# ----------------------- TIM Python Client - Basic Template -----------------------

# 0. Setup
Import the libraries necessary to run this notebook and set up the python client for TIM.
Make sure to fill in your credentials for TIM in the JSON file.

In [14]:
from tim import Tim
import pandas as pd
import json
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go

In [15]:
tim_credentials = json.load(open('tim_credentials.json'))
client = Tim(email=tim_credentials['email'],password=tim_credentials['password'])

# 1. Data Preparation
In this example we use the Belgian Electricity Grid dataset which is already preprocessed for use with TIM.
The dataframe tim_input_df is what will eventually be sent to TIM. If you wish to adapt this file you can apply the necessary transformations here.
The final dataframe is then visualized below in both a graph and a table.

In [16]:
csv_df = pd.read_csv('Belgian Electricity Grid.csv')

In [4]:
tim_input_df = csv_df.copy()

In [5]:
timestamp = 'Timestamp'
target_variable = 'Quantity'
predictor_candidates = [s for s in list(tim_input_df.columns) if s not in [timestamp,target_variable]]
tim_input_df = tim_input_df[[timestamp,target_variable]+predictor_candidates].reset_index(drop=True)

In [ ]:
v_data = tim_input_df
fig = plt.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=v_data[timestamp], y=v_data[target_variable], name=target_variable), row=1, col=1)
for idx, p in enumerate(predictor_candidates):
    fig.add_trace(go.Scatter(x=v_data[timestamp], y=v_data[p], name=p), row=2, col=1)
fig.update_layout(height=600, width=1200, title_text="Data visualization")
fig.show()    

In [7]:
tim_input_df

,Timestamp,Quantity,temp,feels_like,pressure,humidity,wind_speed,wind_deg,rain,snow,clouds,dew_point,wind_gust,visibility,IsPublicHoliday,IsSchoolHoliday,IsHolidayPeriod
0,2021-01-20 00:00:00,9850.0,6.37,-0.83,1005,81,8.23,210,0.0,0.0,75,3.35,0.00,10000.0,0,0,0
1,2021-01-20 01:00:00,9446.0,6.71,-0.59,1005,76,8.23,200,0.0,0.0,75,2.78,0.00,10000.0,0,0,0
2,2021-01-20 02:00:00,9176.0,6.74,-1.11,1004,81,9.26,210,0.5,0.0,100,3.71,0.00,10000.0,0,0,0
3,2021-01-20 03:00:00,9179.0,6.88,-0.95,1003,81,9.26,210,0.0,0.0,0,3.85,0.00,10000.0,0,0,0
4,2021-01-20 04:00:00,9346.0,7.38,0.92,1002,76,7.20,200,0.0,0.0,75,3.43,0.00,10000.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8731,2022-01-18 19:00:00,NaN,3.83,2.07,1035,91,1.94,101,0.0,0.0,5,2.35,2.20,10000.0,0,0,0
8732,2022-01-18 20:00:00,NaN,3.49,3.49,1035,91,1.28,90,0.0,0.0,4,2.03,1.31,10000.0,0,0,0
8733,2022-01-18 21:00:00,NaN,3.26,1.99,1034,91,1.48,89,0.0,0.0,3,1.85,1.48,10000.0,0,0,0
8734,2022-01-18 22:00:00,NaN,3.00,1.83,1034,92,1.39,93,0.0,0.0,4,1.62,1.37,10000.0,0,0,0


# 2. TIM Setup
The configuration below shows you all the options you have for setting up a forecast job with TIM. As you can see there are many possibilities however, TIM will by default already apply quite a lot of settings for you automatically. In this example we only set the "predictionTo" to "24" to generate a 24 sample ahead forecast and we'll set the "outOfSampleRows" to 24*100 = 2400 samples to apply backtesting on the last 100 days of the dataset. All other parameters are commented (#). If not set, TIM will assume default settings.

In [17]:
job_configuration = {
#             "name": "My first forecast job",
#             "useCase": {"id":"useCaseId"},
#             "experiment": {"id":"experimentId"},
            "configuration": {
                "predictionTo": {"baseUnit": "Sample","value": 24},
#                 "predictionFrom": {"baseUnit": "Sample","value": 1},
#                 "modelQuality": "Combined",
#                 "normalization": True,
#                 "maxModelComplexity": 50,
#                 "features": [
#                    "ExponentialMovingAverage",
#                     "RestOfWeek",
#                     "Periodic",
#                     "Intercept",
#                     "PiecewiseLinear",
#                     "TimeOffsets",
#                     "Polynomial",
#                     "Identity",
#                     "PublicHolidays",      
# #                     "SimpleMovingAverage",
# #                     "Month",
# #                     "Trend",
# #                     "DayOfWeek",
# #                     "Fourier",
#                     ],
#                 "dailyCycle": False,
#                 "allowOffsets": True,
#                 "offsetLimit": {"type": "Explicit","value": 0},
#                 "memoryLimitCheck": True,
#                  "predictionIntervals": 90,
#                  "predictionBoundaries": {"type": "Explicit",
#                      "maxValue": 750,
#                     "minValue": 100
#                      },
#                 "rollingWindow": {"baseUnit": "Sample","value": 1},
#                 "backtest": "All"
                },
            "data": {
#                 "version": {"id":"versionId"},
#                 "inSampleRows": {"baseUnit": "Sample","value": 1},
                "outOfSampleRows": {"baseUnit": "Sample","value": 24*100},
#                 "imputation": {"type": "Linear","maxGapLength": 6},
#                 "columns": [
#                     1,
#                     3,
#                     "wind"
#                     ],
#                 "targetColumn": "y",
#                 "holidayColumn": holidayColumn,
#                 "timeScale": {"baseUnit": "Hour","value": 1},
#                 "aggregation": "Mean"
                }
            }

In [18]:
job_configuration

{'configuration': {'predictionTo': {'baseUnit': 'Sample', 'value': 24}},
 'data': {'outOfSampleRows': {'baseUnit': 'Sample', 'value': 2400}}}

In [19]:
def get_forecast_job_results_accuracy(response):   
    bin_json = response['bin']
    bin_accuracy_df = pd.DataFrame()
    for n in bin_json:
        bin_accuracy_df = bin_accuracy_df.append(pd.DataFrame(n).reset_index().rename(columns={'index':'KPI'}))
    bin_accuracy_df['accuracy_type'] = 'bin'

    samplesAhead_json = response['samplesAhead']
    samplesAhead_accuracy_df = pd.DataFrame()
    for n in samplesAhead_json:
        samplesAhead_accuracy_df = samplesAhead_accuracy_df.append(pd.DataFrame(n).reset_index().rename(columns={'index':'KPI'}))
    samplesAhead_accuracy_df['accuracy_type'] = 'samplesAhead'

    all_accuracy_df = pd.DataFrame(response['all']).reset_index().rename(columns={'index':'KPI'})
    all_accuracy_df['accuracy_type'] = 'all'
    id_columns = ['KPI','name','accuracy_type']
    acc_df = all_accuracy_df.append(samplesAhead_accuracy_df).append(bin_accuracy_df)
    df = pd.melt(acc_df, id_vars=id_columns, value_vars=list(set(acc_df.columns)-set(id_columns)))
    return df

def get_forecast_job_results_model(response):
    properties = response['model']['modelZoo']['variableProperties']
    models = response['model']['modelZoo']['models']
    
    pi_df = pd.DataFrame(properties).sort_values(by='importance',ascending=False)
    pi_df['rel_importance'] = pi_df['importance']/pi_df.sum()['importance']
    
    features = []
    for m in models:
        terms = m['terms']
        for count,t in enumerate(terms):
            f,b = find_feature(t['parts'])
            features.append([m['index'],count,f,t['importance'],b])
    fi_df = pd.DataFrame(features,columns=['Model','Term','Feature','importance','beta'])
    return pi_df,fi_df,models

def find_feature(sub_parts):
    dow_list = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    month_list = ['January','February','March','April','May','June','July','August','September','October','November','December']
    features_list = []
    for c,s in enumerate(sub_parts):
        if   s['type']=='β':
            sub_feature = ''
        elif s['type']=='TimeOffsets':
            sub_feature = s['predictor']+'(t'+str(s['offset'])+')'
        elif s['type']=='RestOfWeek':
            sub_feature ='DoW(t'+str(s['offset'])+') <= '+dow_list[s['day']-1]
        elif s['type']=='Intercept':
            sub_feature = 'Intercept('+str(int(s['value']))+')'
        elif s['type']=='Cos':
            sub_feature = 'Cos('+str(int(s['period']))+';'+s['unit']+')'
        elif s['type']=='Sin':
            sub_feature = 'Sin('+str(int(s['period']))+';'+s['unit']+')'
        elif s['type']=='ExponentialMovingAverage':
            sub_feature = 'EMA_'+s['predictor']+'(t'+str(int(s['offset']))+'; w='+str(int(s['window']))+')'
        elif s['type']=='Identity':
            sub_feature = s['predictor']
        elif s['type']=='PiecewiseLinear':
            sub_feature = 'max(0;'+str(s['subtype'])+'*('+str(round(s['knot'],6))+'-'+s['predictor']+'(t'+str(s['offset'])+')))'
        elif s['type']=='SimpleMovingAverage':
            sub_feature = 'SMA_'+s['predictor']+'(t'+str(int(s['offset']))+'; w='+str(int(s['window']))+')'
        elif s['type']=='Fourier':
            sub_feature = 'Fourier('+str(s['period'])+')'
        elif s['type']=='Weekday':
            sub_feature = 'DoW(t'+str(s['offset'])+') = '+dow_list[s['day']-1]
        elif s['type']=='Month':
            sub_feature = 'Month<='+month_list[s['month']]
        elif s['type']=='PublicHoliday':
            sub_feature = s['predictor']
        elif s['type']=='Trend':
            sub_feature = 'Trend'
        else:
            sub_feature = '_test_'
        if s['type']=='β':
            features_list.append(sub_feature)
            beta = s['value']
        else:
            features_list.append(' & '+sub_feature) if c>0 else features_list.append(sub_feature)
    feature_output = ''.join(str(e) for e in features_list)
    return feature_output,beta

# 3. API Call
In this section we communicate with TIM and first upload the dataset, for which you can provide a configuration if you want. Then, the configuration is sent to TIM to create and execute a forecasting job together with the uploaded dataset.

In [11]:
dataset_configuration = {
#     "timestampFormat": "yyyy-mm-dd HH:MM:SS.sss",
#     "timestampColumn": 1,
#     "decimalSeparator": ".",
#     "csvSeparator": ",",
#     "timeZone": "Z",
#     "name": "Vienna",
#     "description": "Electricity consumption",
#     "samplingPeriod": {
#         "baseUnit": "Hour",
#         "value": 1
#     },
#     "workspace": {
#         "id": "ef47117c-5408-4603-9d6f-735f45a74ff3"
#     }
}

In [ ]:
dataset_response = client.upload_dataset(dataset=tim_input_df,configuration=dataset_configuration,handle_status_poll=print)
dataset_id = dataset_response[0]['id']

In [ ]:
forecast_response = client.build_forecasting_model_and_execute(dataset_id=dataset_id,job_configuration=job_configuration,wait_to_finish=True,handle_status_poll=print)

# 4. Collect Results
After the job is finished you can collect your results and insights from TIM using the functions below.
You can use these tables with TIM outputs and adapt them to your needs.

In [40]:
job_meta_data = forecast_response[0]
properties_df,features_df,model = get_forecast_job_results_model(forecast_response[1])
forecast_job_results_table_df = forecast_response[2]
for i in forecast_job_results_table_df['forecast_type'].unique():
    forecast_job_results_table_df.loc[forecast_job_results_table_df['forecast_type']==i, i] = forecast_job_results_table_df['forecast']
accuracy_df = get_forecast_job_results_accuracy(forecast_response[3])
job_logs_df = pd.DataFrame(forecast_response[4]).sort_values(by='createdAt').reset_index(drop=True)

In [41]:
forecast_job_results_table_df['MAE'] = abs(forecast_job_results_table_df['forecast']-forecast_job_results_table_df['target'])

# 5. Visualize Results
Images below show how TIM results can be easily visualized in Python with Plotly. You can visualize the predictions, the accuracy values calculated by TIM, the predictor and feature importances and other insights.

In [ ]:
v_data = forecast_job_results_table_df
fig = plt.subplots.make_subplots(rows=2, cols=1, vertical_spacing=0.04,shared_xaxes=True)
fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['target'], name='Actuals', line=dict(color='black')), row=1, col=1)
fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['Production'], name='Prediction', line=dict(color='goldenrod')), row=1, col=1)
# fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['lower_bound'], name='Lower bound', line=dict(color='lightgrey')), row=1, col=1)
# fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['upper_bound'], name='Upper bound', line=dict(color='lightgrey')), row=1, col=1)
fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['InSample'], name='InSample', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['OutOfSample'], name='OutOfSample', line=dict(color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=v_data['timestamp'], y=v_data['MAE'], name='MAE', line=dict(color='blue')), row=2, col=1)
fig.update_layout(height=800, width=1200, title_text="Results")
fig.show()

In [ ]:
v_data = forecast_job_results_table_df[['target','OutOfSample']].dropna()
x_axis = 'target'
y_axis = 'OutOfSample'
max_x = max(v_data[x_axis].max(),v_data[y_axis].max())
min_x = min(v_data[x_axis].min(),v_data[y_axis].min())
fig = go.Figure()
fig.add_trace(go.Scatter(x=v_data[x_axis], y=v_data[y_axis],mode='markers',name='results'))
fig.add_trace(go.Scatter(x=[min_x,max_x], y=[min_x,max_x],mode='lines',name='y=x'))
fig.update_layout(height=500, width=1200, title_text="Parity Plot")
fig.show()

In [ ]:
rows = accuracy_df.dropna()['accuracy_type'].unique()
columns = accuracy_df.dropna()['KPI'].unique()
subplot_titles = []
for i in rows:
    for j in columns: subplot_titles.append(str(i)+" "+str(j))
fig = plt.subplots.make_subplots(rows=len(rows), cols=len(columns), vertical_spacing=0.04,subplot_titles=subplot_titles)

for r,i in enumerate(rows):
    for c,j in enumerate(columns):
        v_data = accuracy_df[(accuracy_df['KPI']==j)&(accuracy_df['accuracy_type']==i)].pivot(index=['KPI','name','accuracy_type'], columns='variable', values='value').reset_index()
        fig.add_trace(go.Bar(x=v_data['name'], y=v_data['inSample'], name=str(i)+" "+str(j),text=round(v_data['inSample'],2),textposition='auto'), row=r+1, col=c+1)
        fig.add_trace(go.Bar(x=v_data['name'], y=v_data['outOfSample'], name=str(i)+" "+str(j),text=round(v_data['outOfSample'],2),textposition='auto'), row=r+1, col=c+1)
fig.update_layout(height=1200, width=1400, title_text="Data visualization",)
fig.show()

In [ ]:
b_v_df = properties_df[properties_df['importance']>0]
x_axis = 'name'
y_axis = 'rel_importance'

fig1 = go.Figure(go.Bar(x=b_v_df[x_axis], y=b_v_df[y_axis],text=round(b_v_df[y_axis],2),textposition='auto'))
fig1.update_layout(height=500,width=1200,title_text='Predictor Importances',xaxis_title=x_axis,yaxis_title=y_axis)
print('Predictors not used:'+str(list(properties_df[~(properties_df['importance']>0)]['name'])))
fig1.show()

In [ ]:
fig = px.sunburst(features_df, path=['Model','Feature'], values='importance',color='Feature')
fig.update_layout(height=700,width=700,title_text='Feature Importances')
fig.show()

In [ ]:
job_logs_df

In [48]:
warnings = list(job_logs_df[job_logs_df['messageType'] == "Warning"]['message'])
warnings

[]

# 6. Save Results
To save the outputs from TIM use the cells below. You can also find a summary of all the other functions in the Python Client for TIM below. 
For more information visit https://pypi.org/project/tim-client/ or our documentation https://docs.tangent.works/TIM-on-Platforms/Python-Client/Overview/.

In [49]:
# forecast_job_results_table_df.to_csv('forecast_job_results_table_df.csv',index=False, float_format='%g')
# accuracy_df.to_csv('accuracy_df.csv',index=False, float_format='%g')
# properties_df.to_csv('properties_df.csv',index=False, float_format='%g')
# features_df.to_csv('features_df.csv',index=False, float_format='%g')
# job_logs_df.to_csv('job_logs_df.csv',index=False, float_format='%g')

In [22]:
# client.upload_dataset
# client.delete_dataset
# client.get_datasets
# client.get_dataset_versions
# client.build_forecasting_model
# client.execute_forecast
# client.build_forecasting_model_and_execute
# client.clean_forecast
# client.get_forecast_results
# client.get_forecasting_jobs
# client.delete_forecast
# client.build_anomaly_detection_model
# client.execute_anomaly_detection
# client.build_anomaly_detection_model_and_execute
# client.create_anomaly_detection
# client.create_anomaly_detection_and_execute
# client.get_anomaly_detection_results
# client.get_anomaly_detection_jobs
# client.delete_anomaly_detection
# client.get_workspaces